In [42]:
import dataverk as dv
from dataverk_tools.themes import nav

import pandas as pd
import ipywidgets as widgets

import plotly.graph_objs as go
import plotly.io as plio
import plotly_express as px
import plotly.figure_factory as ff

from datetime import datetime

In [ ]:
df = pd.read_csv('https://storage.googleapis.com/nav-opendata/stillinger_all_merged.csv.gz',sep=';',low_memory=False)

In [43]:
len(df)

2603279

In [44]:
df.dtypes

stillingsnummer                        object
stillingsnummer_nav_no                 object
stilling_id                             int64
nav_enhet_kode                         object
registrert_dato                        object
sistepubl_dato                         object
statistikk_aar_mnd                      int64
offisiell_statistikk_flagg              int64
stilling_kilde                         object
arbeidssted_fylkesnummer               object
arbeidssted_fylke                      object
arbeidssted_kommunenummer              object
arbeidssted_kommune                    object
arbeidssted_landkode                   object
arbeidssted_land                       object
isco_versjon                           object
yrke_grovgruppe                        object
yrkeskode                               int64
yrke                                   object
yrkesbetegnelse                        object
bedrift_org_nr                         object
bedrift_navn                      

In [32]:
df['tid'] = df['statistikk_aar_mnd'].apply(lambda x : datetime.strptime(str(x), '%Y%m'))

In [33]:
df['yrke_grovgruppe_merged'] = df['yrke_grovgruppe'].apply(lambda x : x.replace(' (2001-2011)',''))

In [34]:
#nav.plotly_template

In [35]:
df_c = df[['tid','stillingsnummer']].groupby('tid').count().reset_index()

In [45]:
fig_antall_stillinger = px.line(df_c, x='tid', y='stillingsnummer', template=nav.plotly_template)
fig_antall_stillinger

In [37]:
df_k = df[['stillingsnummer','arbeidssted_fylke']].groupby('arbeidssted_fylke').count().reset_index() \
    .sort_values(by='stillingsnummer', ascending=False)
fig_foretak_arbeidssted_fylke = px.bar(df_k, x='arbeidssted_fylke', y='stillingsnummer', template=nav.plotly_template)
fig_foretak_arbeidssted_fylke

In [38]:
df_k = df[['stillingsnummer','foretak_sektor_gruppe']].groupby('foretak_sektor_gruppe').count().reset_index() \
    .sort_values(by='stillingsnummer', ascending=False)
fig_foretak_sektor_gruppe = px.bar(df_k, x='foretak_sektor_gruppe', y='stillingsnummer', template=nav.plotly_template)
fig_foretak_sektor_gruppe

In [39]:
df_k = df[['stillingsnummer','stilling_kilde']].groupby('stilling_kilde').count().reset_index() \
    .sort_values(by='stillingsnummer', ascending=False)
fig_stilling_kilde = px.bar(df_k, x='stilling_kilde', y='stillingsnummer', template=nav.plotly_template)
fig_stilling_kilde

In [40]:
df_k = df[['stillingsnummer','bedrift_navn']].groupby('bedrift_navn').count().reset_index() \
    .sort_values(by='stillingsnummer', ascending=False).head(60)
fig_bedrift_navn = px.bar(df_k, x='bedrift_navn', y='stillingsnummer', template=nav.plotly_template)
fig_bedrift_navn

# Datapakke

In [56]:
readme = """
# Ledige stillinger annonsert på nav.no
"""

metadata = {
'store': 'gs',
'repo': 'https://raw.githubusercontent.com/datasett/nav-examples/master/',
'readme': readme,
'license':'MIT',
'accessRights':'Open',
'author': 'paul.bencze@nav.no',
'description':'Ledige stillinger annonsert på nav.no',
'name':'Ledige stillinger annonsert på nav.no',
'source':'NAV',
'keywords':['stillinger, annonser,sysselsetting,yrker'],
'provenance':'NAV',
'publisher': 'NAV',
'bucket': 'nav-opendata',
}

In [57]:
dp = dv.Datapackage(metadata)

In [58]:
dp.add_resource(df,'Stillingsannonser m/tekst','Stillingsannonser m/tekst')

In [59]:
description = '''
Antall stillinger annonsert på nav.no pr måned (statistikkmåned - her kommer en forklaring av hva vi mener med statistikkmåned)
'''

dp.add_view(
    spec_type='plotly',
    name='Antall stillinger annonsert pr mnd',
    title='Antall stillinger annonsert pr mnd',
    description = description,
    resources=[],
    attribution = "Kilde: NAV.",
    spec=plio.to_json(fig_antall_stillinger)
)

In [60]:
description = '''
Antall stillinger annonsert på nav.no fordelt på fylke (arbeidssted)
'''

dp.add_view(
    spec_type='plotly',
    name='Antall stillinger fordelt på fylke',
    description = description,
    resources=[],
    attribution = "Kilde: NAV.",
    spec=plio.to_json(fig_foretak_arbeidssted_fylke)
)

In [61]:
description = '''
Antall stillinger annonsert på nav.no fordelt på sektor
'''

dp.add_view(
    spec_type='plotly',
    name='Antall stillinger annonsert fordelt på sektor',
    description = description,
    resources=[],
    attribution = "Kilde: NAV.",
    spec=plio.to_json(fig_foretak_sektor_gruppe)
)

In [62]:
description = '''
Antall stillinger annonsert på nav.no fordelt på yrke (grovgruppe)
'''

dp.add_view(
    spec_type='plotly',
    name='Antall stillinger annonsert fordelt på yrke (grovgruppe)',
    description = description,
    resources=[],
    attribution = "Kilde: NAV.",
    spec=plio.to_json(fig_yrke_grovgruppe)
)

In [63]:
description = '''
Antall stillinger annonsert på nav.no fordelt på kilde for annonsen
'''

dp.add_view(
    spec_type='plotly',
    name='Antall stillinger fordelt på kilde',
    description = description,
    resources=[],
    attribution = "Kilde: NAV.",
    spec=plio.to_json(fig_stilling_kilde)
)

In [64]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/users/b149899/projects/binder/binder/keys/nav_opendata.json'

In [65]:
client = dv.Client()
client.publish(dp)

gs
2019-06-16T21:42:36.857204: B149899 <class 'dataverk.connectors.google_storage.GoogleStorageConnector'>: String (format: json) written to https://storage.googleapis.com/nav-opendata/78dabb4c62d8a74e1d56fbed1a336e42/datapackage.json
2019-06-16T22:02:01.155545: B149899 <class 'dataverk.connectors.google_storage.GoogleStorageConnector'>: String (format: csv) written to https://storage.googleapis.com/nav-opendata/78dabb4c62d8a74e1d56fbed1a336e42/resources/Stillingsannonser%20m/tekst.csv
2019-06-16T22:02:01.824412: B149899 <class 'dataverk.connectors.elasticsearch.ElasticsearchConnector'>: Document 78dabb4c62d8a74e1d56fbed1a336e42 of type odata indexed to elastic index: odata.
